<a href="https://colab.research.google.com/github/ssv273/Neural_Univesity/blob/main/hw_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential #Сеть прямого распространения
#Базовые слои для счёрточных сетей
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator # работа с изображениями
from tensorflow.keras.optimizers import Adam, Adadelta # оптимизаторы
from tensorflow.keras import utils #Используем дял to_categoricall
from tensorflow.keras.preprocessing import image #Для отрисовки изображений
from google.colab import files #Для загрузки своей картинки
import numpy as np #Библиотека работы с массивами
import matplotlib.pyplot as plt #Для отрисовки графиков
from PIL import Image #Для отрисовки изображений
import random #Для генерации случайных чисел 
import math # Для округления
import os #Для работы с файлами 
# подключем диск
from google.colab import drive
from tensorflow.keras.callbacks import ModelCheckpoint

%matplotlib inline

In [ ]:
#!unzip "/content/drive/MyDrive/cars.zip" -d /content/cars #Указываем путь к базе в Google Drive

In [ ]:
os.listdir('cars')

['Автомобили']

In [ ]:
train_path = '/content/cars/Автомобили' #Папка с папками картинок, рассортированных по категориям
batch_size = 25 #Размер выборки
img_width = 96 #Ширина изображения
img_height = 54 #Высота изображения

In [ ]:
#Генератор изображений
datagen = ImageDataGenerator(
    rescale=1. / 255, #Значения цвета меняем на дробные показания
    rotation_range=10, #Поворачиваем изображения при генерации выборки
    width_shift_range=0.1, #Двигаем изображения по ширине при генерации выборки
    height_shift_range=0.1, #Двигаем изображения по высоте при генерации выборки
    zoom_range=0.1, #Зумируем изображения при генерации выборки
    horizontal_flip=True, #Включаем отзеркаливание изображений
    fill_mode='nearest', #Заполнение пикселей вне границ ввода
    validation_split=0.2 #Указываем разделение изображений на обучающую и тестовую выборку
)

In [ ]:
# обучающая выборка
train_generator = datagen.flow_from_directory(
    train_path, #Путь ко всей выборке выборке
    target_size=(img_width, img_height), #Размер изображений
    batch_size=batch_size, #Размер batch_size
    class_mode='categorical', #Категориальный тип выборки. Разбиение выборки по маркам авто 
    shuffle=True, #Перемешивание выборки
    subset='training' # устанавливаем как набор для обучения
)

# проверочная выборка
validation_generator = datagen.flow_from_directory(
    train_path, #Путь ко всей выборке выборке
    target_size=(img_width, img_height), #Размер изображений
    batch_size=batch_size, #Размер batch_size
    class_mode='categorical', #Категориальный тип выборки. Разбиение выборки по маркам авто 
    shuffle=True, #Перемешивание выборки
    subset='validation' # устанавливаем как валидационный набор
)

Found 2743 images belonging to 2 classes.
Found 684 images belonging to 2 classes.


In [ ]:
#Создаем последовательную модель
model = Sequential()
#Первый сверточный слой
model.add(Conv2D(256, (3, 3), padding='same', activation='relu', input_shape=(img_width, img_height, 3)))
#Второй сверточный слой
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
#Третий сверточный слой
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
#Слой регуляризации Dropout
model.add(Dropout(0.2))
#Четвертый сверточный слой
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
#Слой регуляризации Dropout
model.add(Dropout(0.2))
#Пятый сверточный слой
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
#Шестой сверточный слой
model.add(Conv2D(1024, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
#Слой регуляризации Dropout
model.add(Dropout(0.2))
#Слой преобразования двумерных данных в одномерные 
model.add(Flatten())
#Полносвязный слой
model.add(Dense(4096, activation='relu'))
#Полносвязный слой
model.add(Dense(2048, activation='relu'))
#Вызодной полносвязный слой
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
filepath = 'Checkpoints/my_best_model.epoch{epoch:02d}-accuracy{val_accuracy:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_accuracy',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

In [ ]:
callbacks = [checkpoint]

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs=45,
    verbose=1,
    callbacks=callbacks
)

In [ ]:
model.load_weights('/content/Checkpoints/my_best_model.epoch07-accuracy0.81.hdf5')

In [ ]:
from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.00001)

In [ ]:
K.eval(model.optimizer.lr)

1e-05

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs=45,
    verbose=1,
    callbacks=callbacks
)

Epoch 1/45
109/109 [==============================] - 13s 120ms/step - loss: 0.4889 - accuracy: 0.8028 - val_loss: 0.5032 - val_accuracy: 0.8000

Epoch 00001: val_accuracy did not improve from 0.80889
Epoch 2/45
109/109 [==============================] - 13s 119ms/step - loss: 0.4902 - accuracy: 0.7999 - val_loss: 0.5034 - val_accuracy: 0.8000

Epoch 00002: val_accuracy did not improve from 0.80889
Epoch 3/45
109/109 [==============================] - 13s 118ms/step - loss: 0.4877 - accuracy: 0.8010 - val_loss: 0.5044 - val_accuracy: 0.7985

Epoch 00003: val_accuracy did not improve from 0.80889
Epoch 4/45
109/109 [==============================] - 13s 117ms/step - loss: 0.4880 - accuracy: 0.8002 - val_loss: 0.5030 - val_accuracy: 0.8000

Epoch 00004: val_accuracy did not improve from 0.80889
Epoch 5/45
109/109 [==============================] - 13s 119ms/step - loss: 0.4873 - accuracy: 0.7995 - val_loss: 0.5023 - val_accuracy: 0.8030

Epoch 00005: val_accuracy did not improve from 0.8

# ResNet

In [ ]:
EXP_NAME = 'transfer'
base_model = tf.keras.applications.ResNet152V2(
    input_shape=(img_width, img_height, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable=True

In [ ]:
model_resnet = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')                     
])

In [ ]:
model_resnet.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
filepath = 'Checkpoints/my_best_model_resnet.epoch{epoch:02d}-accuracy{val_accuracy:.2f}.hdf5'
checkpoint_resnet = ModelCheckpoint(filepath=filepath, 
                             monitor='val_accuracy',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

In [ ]:
history = model_resnet.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs=45,
    verbose=1, 
    callbacks=[checkpoint_resnet]
)

Epoch 1/45
109/109 [==============================] - 39s 219ms/step - loss: 0.6087 - accuracy: 0.7428 - val_loss: 1.0527 - val_accuracy: 0.7881

Epoch 00001: val_accuracy improved from -inf to 0.78815, saving model to Checkpoints/my_best_model_resnet.epoch01-accuracy0.79.hdf5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/45
109/109 [==============================] - 20s 183ms/step - loss: 0.5180 - accuracy: 0.7914 - val_loss: 0.7644 - val_accuracy: 0.7733

Epoch 00002: val_accuracy did not improve from 0.78815
Epoch 3/45
109/109 [==============================] - 20s 179ms/step - loss: 0.4844 - accuracy: 0.7943 - val_loss: 0.6477 - val_accuracy: 0.7600

Epoch 00003: val_accuracy did not improve from 0.78815
Epoch 4/45
109/109 [==============================] - 19s 177ms/step - loss: 0.4480 - accuracy: 0.8043 - val_loss: 0.6942 - val_accuracy: 0.7837

Epoch 00004: val_accuracy did not improve from 0.78815
Epoch 5/45
109/109 [==============================] - 20s 179ms/step - loss: 0.4364 - accuracy: 0.8127 - val_loss: 0.6322 - val_accuracy: 0.7926

Epoch 00005: val_accuracy improved from 0.78815 to 0.79259, saving model to Checkpoints/my_best_model_resnet.epoch05-accuracy0.79.hdf5
Epoch 6/45
109/109 [==============================] - 19s 178ms/step - loss: 0.4228 - accuracy: 0.8230 - val_loss: 0